In [1]:
def isMatch(s: str, p: str) -> bool:
    m = len(s)
    n = len(p)
    dp = [[False] * (n + 1) for _ in range(m + 1)]
    dp[0][0] = True
    
    for j in range(1, n + 1):
        if p[j-1] == '*':
            dp[0][j] = dp[0][j-1]
            
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if p[j-1] == s[i-1] or p[j-1] == '?':
                dp[i][j] = dp[i-1][j-1]
            elif p[j-1] == '*':
                # dp[i-1][j]: '*' matches s[i-1]
                # dp[i][j-1]: '*' matches empty sequence
                dp[i][j] = dp[i-1][j] or dp[i][j-1]
            else:
                dp[i][j] = False
                
    return dp[m][n]

The LeetCode problem 44, "Wildcard Matching," is a string matching challenge where the pattern $p$ can contain two special wildcard characters: '?' and '*'. The goal is to determine if the pattern $p$ matches the entire input string $s$. The '?' character matches any single character, while the '*' character matches any sequence of characters, including an empty sequence. This problem is similar to regular expression matching but with a simplified set of wildcards.

---

### **The Fundamental Challenge: The Ambiguity of the Asterisk ('*')**

The primary difficulty in this problem comes from the '*' wildcard. When an asterisk is encountered in the pattern, it can potentially match zero, one, or multiple characters in the input string $s$. This creates a branching decision tree at every step, making a simple linear traversal insufficient. For example, if $s = \text{"adceb"}$ and $p = \text{"a*b"}$, the '*' can match the empty sequence, $\text{"d"}$, $\text{"dc"}$, or $\text{"dce"}$. The solution must explore all these possibilities efficiently.

---

### **Approach 1: Dynamic Programming (DP)**

The most common and structurally sound solution is Dynamic Programming. We define a 2D boolean array, $dp$, where $dp[i][j]$ is `True` if the first $i$ characters of the string $s$ match the first $j$ characters of the pattern $p$. The size of the DP table is $(N+1) \times (M+1)$, where $N = \text{length}(s)$ and $M = \text{length}(p)$.

1.  **Initialization:** $dp[0][0]$ is `True` (empty string matches empty pattern). $dp[i][0]$ is `False` for $i > 0$ (non-empty string cannot match an empty pattern). For $p$ with leading asterisks, $dp[0][j]$ is `True` if $p[0...j-1]$ consists only of asterisks, reflecting that '*' can match an empty string.

2.  **Transition Logic (Character Match/Single Wildcard):** If $p[j-1]$ is a literal character or '?', then $dp[i][j]$ is `True` if and only if $s[i-1] == p[j-1]$ (or $p[j-1] == '?'$) **and** $dp[i-1][j-1]$ is `True`. This is the standard diagonal transition.

3.  **Transition Logic (Asterisk '*'):** If $p[j-1] = '*'$, it can match three possibilities:
    * **Empty Sequence (0 match):** The '*' matches nothing. We check if the match holds without the asterisk: $dp[i][j-1]$.
    * **One or More Characters (1+ match):** The '*' matches the current character $s[i-1]$, and potentially more. We check if the match holds for the previous character in $s$: $dp[i-1][j]$.

    Therefore, if $p[j-1] = '*'$, $dp[i][j] = dp[i][j-1] \lor dp[i-1][j]$. 

The final result is $dp[N][M]$. The time and space complexity for this approach are both $O(N \cdot M)$.

---

### **Approach 2: Greedy Backtracking with Two Pointers (The Optimal Space Solution)**

The DP solution can be optimized to $O(1)$ space using a **Greedy Two-Pointer** approach with explicit backtracking for the asterisk. This approach leverages the fact that we only need two pointers for $s$ and $p$ and two pointers to save the last successful asterisk match position for backtracking.

1.  **Pointers and Anchors:** We use $i$ and $j$ to iterate through $s$ and $p$. We also use two special variables:
    * `star_index`: Stores the index in $p$ where the last '*' was encountered.
    * `s_save_index`: Stores the index in $s$ that corresponds to the first character the last '*' *didn't* match. This is the position to which we backtrack.

2.  **Iteration Logic:** The main loop continues while $i < N$.
    * **Exact Match or '?':** If $s[i] = p[j]$ or $p[j] = '?'$, advance both $i$ and $j$.
    * **Asterisk '*':** If $p[j] = '*'$, save the asterisk's position: set `star_index = j` and `s_save_index = i`. Then, advance $j$. (This is the greedy choice: assume '*' matches the empty string for now).
    * **Mismatch and Backtrack:** If a mismatch occurs and there was a previously saved `star_index` (an asterisk is available for use):
        * **Backtrack:** Set $j = star\_index + 1$ (use the asterisk to match the current $s[i]$ and resume $p$ after it).
        * **Advance $s$:** Set $i = s\_save\_index + 1$ (make the asterisk match one more character in $s$).
        * **Update Save Point:** Update `s_save_index` to the new starting point for the asterisk: $s\_save\_index = i$.
    * **Mismatch and No Backtrack:** If a mismatch occurs and there is no available asterisk (`star\_index = -1`), the match fails. Return `False`.

3.  **Final Pattern Check:** After the main loop finishes (when $s$ is fully consumed, $i=N$), we must check if the remaining characters in $p$ are all asterisks. If $p$ has trailing characters that are not '*', the match fails.

---

### **Complexity Analysis**

* **Time Complexity:** The greedy approach appears complex, but since both $i$ and $j$ advance almost linearly, and backtracking only occurs when an asterisk is involved, the total number of character comparisons is still bounded by $O(N \cdot M)$ in the worst case (though often much faster in practice).
* **Space Complexity:** The greedy approach only uses a fixed number of pointers and indices, achieving an optimal $O(1)$ auxiliary space complexity.